In [1]:
import math
import numpy as np

In [2]:
EPS = 1e-7
ITERATIONS = 200000

In [ ]:
def _format_matrix(min, max, size):
    return np.array((max - min) * np.random.random((size, size)) + min, dtype=np.float64)
    
def rand_matrix(size):
    A = np.array(np.random.rand(size, size), dtype=np.float64)
    A = np.tril(A) + np.tril(A, -1).T
    return A.T * A

def diag_matrix(size):
    A = _format_matrix(0.0, 10.0, size)
    for i in range(size):
        A[i, i] = sum(A[i]) + 1
    return A

def hilbert_matrix(size):
    ans = np.array(np.zeros((size, size)), dtype=np.float64)
    for i in range(size):
        for j in range(size):
            ans[i, j] = 1.0 / (i + j + 1)
    return ans

def gen_right_column(size):
    return np.array(np.random.random(size), dtype=np.float64)

In [4]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [20]:
def zeidelMethod(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            
    print("Iterations = ", curIterations)
    return xn

In [73]:
def zeidelMethodRelax(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    Relax = 1.6
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            xn[i] = Relax * xn[i] + (1 - Relax) * x[i]
            
    print("Iterations = ", curIterations)
    return xn

In [74]:
def _test(method, matrix, column):
    return method(matrix, column)

def _check_eps(a, b, eps):
    return abs(a - b) < eps
EPSILON = 1e-2
def _check_ans(matrix, column, res, size):
    eps = EPSILON
    c = np.matmul(matrix, res)
    for i in range(size):
        if not _check_eps(c[i], column[i], eps):
            return False
    return True

def _testAll(matrix_gen, size, methods):
    matrix, column = matrix_gen(size), gen_right_column(size)
    print(f'Тест для матрицы {matrix} и столбца {column}')
    for method in methods:
        print(f'Результат выполнения метода {method.__name__}:')
        res = _test(method, matrix, column)
        check = _check_ans(matrix, column, res, size)
        if check:
            print(f'{res}. Погрешность не больше {EPSILON}\n')
        else:
            print(f'разошлось: matrix * res = {np.matmul(matrix, res)}\n ожидаемый столбец = {column}\n')

def _get_methods():
    return [zeidelMethod, zeidelMethodRelax]

def testDiagMatrix(size):
    _testAll(diag_matrix, size, _get_methods())

def testHilbertMatrix(size):
    _testAll(hilbert_matrix, size, _get_methods())
    
def testRandMatrix(size):
    _testAll(rand_matrix, size, _get_methods())

In [75]:
testDiagMatrix(10)

Тест для матрицы [[3.46890717e+01 2.33556386e+00 8.00040469e+00 1.54554586e+00
  2.03009794e+00 4.63366698e-01 3.29695873e+00 1.42605804e+00
  6.36913722e+00 2.52800423e+00]
 [3.24138575e+00 5.75260190e+01 5.39899842e+00 1.25824561e+00
  7.27903675e+00 9.04181699e+00 4.01524295e+00 6.84580062e+00
  9.05782692e+00 8.69885514e+00]
 [3.99109753e+00 4.67542048e+00 4.25960505e+01 2.99400908e+00
  2.14780156e+00 1.63516936e+00 5.48483891e+00 7.71846003e+00
  3.23987534e+00 9.49278037e+00]
 [8.71843237e+00 2.47026669e-01 9.67041985e+00 6.45904589e+01
  5.01792430e+00 8.71280040e+00 1.75672424e+00 7.08620501e+00
  8.60313648e+00 9.09333227e+00]
 [5.39056953e+00 9.05586589e+00 7.21343471e+00 6.27719681e+00
  5.70987537e+01 5.57372832e-01 6.15339364e+00 7.31334623e+00
  1.94689958e-02 5.43803320e+00]
 [2.74355931e+00 7.56134401e+00 9.46125228e+00 3.87082407e+00
  6.15552507e-02 4.86031517e+01 9.71104221e-01 5.58320739e+00
  5.97228989e+00 9.93301620e+00]
 [4.86155229e+00 9.52618703e+00 1.5318094

/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in double_scalars
  if sys.path[0] == '':
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


Iterations =  200000
разошлось: matrix * res = [nan nan nan nan nan nan nan nan nan nan]
 ожидаемый столбец = [0.43979492 0.60547122 0.17596533 0.07472778 0.46759727 0.33105511
 0.22034808 0.47161914 0.28709814 0.15128231]



In [76]:
testRandMatrix(4)

Тест для матрицы [[0.47346342 0.00345918 0.93434058 0.12272202]
 [0.00345918 0.12549191 0.03013767 0.69972778]
 [0.93434058 0.03013767 0.66749258 0.42563132]
 [0.12272202 0.69972778 0.42563132 0.07106464]] и столбца [0.00808198 0.54414593 0.45174299 0.63473201]
Результат выполнения метода zeidelMethod:


/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/kranya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


KeyboardInterrupt: 

In [ ]:
testHilbertMatrix(5)